In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df=pd.read_csv("/content/drive/MyDrive/Internship/kaggle/kaggle_visible_evaluation_triplets.txt",sep='\t',names=["user_id",'song_id','freq'])

In [5]:
df.shape

(1450933, 3)

In [6]:
df.head()

,user_id,song_id,freq
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1


In [7]:
unique=pd.read_csv("/content/drive/MyDrive/Internship/kaggle/unique_tracks.txt",sep='<SEP>',
                   names=["track_id","song_id","artist_name","release"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [8]:
unique.head(5)

,track_id,song_id,artist_name,release
0,TRMMMYQ128F932D901,SOQMMHC12AB0180CB8,Faster Pussy cat,Silent Night
1,TRMMMKD128F425225D,SOVFVAK12A8C1350D9,Karkkiautomaatti,Tanssi vaan
2,TRMMMRX128F93187D9,SOGTUKN12AB017F4F1,Hudson Mohawke,No One Could Ever
3,TRMMMCH128F425532C,SOBNYVR12A8C13558C,Yerba Brava,Si Vos Querés
4,TRMMMWA128F426B589,SOHSBXH12A8C13B0DF,Der Mystic,Tangle Of Aspens


In [9]:
unique.shape

(1000000, 4)

In [10]:
unique.drop_duplicates(subset='song_id',keep='first',inplace=True)

In [11]:
final_data=pd.merge(df,unique,how='left',on='song_id')
final_data.head()

,user_id,song_id,freq,track_id,artist_name,release
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,TRAEHHJ12903CF492F,Dwight Yoakam,You're The One
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1,TRLGMFJ128F4217DBE,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1,TRTNDNE128F1486812,Cartola,Tive Sim
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1,TRASTUE128F930D488,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1,TRFPLWO128F1486B9E,Miguel Calo,El Cuatrero


In [12]:
final_data.shape

(1450933, 6)

**ITEM BASED RECOMMENDATION**

In [13]:
 class item_similarity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.cooccurence_matrix = None
        self.songs_dict = None
        self.rev_songs_dict = None
        self.item_similarity_recommendations = None
        
    #Get unique items (songs) corresponding to a given user
    def get_user_items(self, user):
        user_data = self.train_data[self.train_data[self.user_id] == user]
        user_items = list(user_data[self.item_id].unique())
        
        return user_items
        
    #Get unique users for a given item (song)
    def get_item_users(self, item):
        item_data = self.train_data[self.train_data[self.item_id] == item]
        item_users = set(item_data[self.user_id].unique())
            
        return item_users
        
    #Get unique items (songs) in the training data
    def get_all_items_train_data(self):
        all_items = list(self.train_data[self.item_id].unique())
            
        return all_items
        
    #Construct cooccurence matrix
    def construct_cooccurence_matrix(self, user_songs, all_songs):
        #Get users for all songs in user_songs.
        user_songs_users = []        
        for i in range(0, len(user_songs)):
            user_songs_users.append(self.get_item_users(user_songs[i]))
        #Initialize the item cooccurence matrix of size len(user_songs) X len(songs)
        cooccurence_matrix = np.matrix(np.zeros(shape=(len(user_songs), len(all_songs))), float)
        #Calculate similarity between user songs and all unique songs in the training data
        for i in range(0,len(all_songs)):
            #Calculate unique listeners (users) of song (item) i
            songs_i_data = self.train_data[self.train_data[self.item_id] == all_songs[i]]
            users_i = set(songs_i_data[self.user_id].unique())
            
            for j in range(0,len(user_songs)):       
                    
                #Get unique listeners (users) of song (item) j
                users_j = user_songs_users[j]
                    
                #Calculate intersection of listeners of songs i and j
                users_intersection = users_i.intersection(users_j)
                
                #Calculate cooccurence_matrix[i,j] as Jaccard Index
                if len(users_intersection) != 0:
                    #Calculate union of listeners of songs i and j
                    users_union = users_i.union(users_j)
                    
                    cooccurence_matrix[j,i] = float(len(users_intersection))/float(len(users_union))
                else:
                    cooccurence_matrix[j,i] = 0
                    
        
        return cooccurence_matrix

    
    #Use the cooccurence matrix to make top recommendations
    def generate_top_recommendations(self, user, cooccurence_matrix, all_songs, user_songs):
        print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(cooccurence_matrix))
        
        #Calculate a weighted average of the scores in cooccurence matrix for all user songs.
        user_sim_scores = cooccurence_matrix.sum(axis=0)/float(cooccurence_matrix.shape[0])
        user_sim_scores = np.array(user_sim_scores)[0].tolist()
 
        #Sort the indices of user_sim_scores based upon their value
        #Also maintain the corresponding score
        sort_index = sorted(((e,i) for i,e in enumerate(list(user_sim_scores))), reverse=True)
    
        #Create a dataframe from the following
        columns = ['user_id', 'song', 'score', 'rank']
        #index = np.arange(1) # array of numbers for the number of samples
        df = pd.DataFrame(columns=columns)
         
        #Fill the dataframe with top 10 item based recommendations
        rank = 1 
        for i in range(0,len(sort_index)):
            if ~np.isnan(sort_index[i][0]) and all_songs[sort_index[i][1]] not in user_songs and rank <= 10:
                df.loc[len(df)]=[user,all_songs[sort_index[i][1]],sort_index[i][0],rank]
                rank = rank+1
        
        #Handle the case where there are no recommendations
        if df.shape[0] == 0:
            print("The current user has no songs for training the item similarity based recommendation model.")
            return -1
        else:
            return df
 
    #Create the item similarity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

    #Use the item similarity based recommender system model to
    #make recommendations
    def recommend(self, user):
        #Get all unique songs for this user
        user_songs = self.get_user_items(user)
        print("No. of unique songs for the user: %d" % len(user_songs))
        all_songs = self.get_all_items_train_data()
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        #D. Use the cooccurence matrix to make recommendations
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
                
        return df_recommendations
    
    #Get similar items to given items
    def get_similar_items(self, item_list):
        user_songs = item_list
        #B. Get all unique items (songs) in the training data
        all_songs = self.get_all_items_train_data()

        print("no. of unique songs in the training set: %d" % len(all_songs))
        #C. Construct item cooccurence matrix of size 
        #len(user_songs) X len(songs)
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)
        #D. Use the cooccurence matrix to make recommendations
        user = ""
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)
         
        return df_recommendations

## **Creating a sample dataset for recommendation**

In [14]:
  final_sample = final_data.head(5000)

In [15]:
final_sample

,user_id,song_id,freq,track_id,artist_name,release
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,TRAEHHJ12903CF492F,Dwight Yoakam,You're The One
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1,TRLGMFJ128F4217DBE,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1,TRTNDNE128F1486812,Cartola,Tive Sim
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1,TRASTUE128F930D488,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1,TRFPLWO128F1486B9E,Miguel Calo,El Cuatrero
...,...,...,...,...,...,...
4995,13cee37ea2e198ce8354a962b118c1770db351d6,SOBFPEX12A58A7A698,1,TRWSOZK12903C9554D,A Hawk And A Hacksaw,Portlandtown
4996,13cee37ea2e198ce8354a962b118c1770db351d6,SOOKGRB12A8C13CD66,1,TRYHWLD128F426BF7D,Fleet Foxes,Mykonos
4997,13cee37ea2e198ce8354a962b118c1770db351d6,SOGLSPG12A8C13FD9E,1,TRTNODY128F42B772F,PJ Harvey,Bows & Arrows
4998,13cee37ea2e198ce8354a962b118c1770db351d6,SOUFBTM12A8AE46153,1,TRRMCOW128F4259DF0,My Bloody Valentine,(When You Wake) You're Still In A Dream


In [30]:
recommendation = item_similarity_recommender_py()
recommendation.create(final_sample, 'user_id', 'release')

In [31]:
user_items = recommendation.get_user_items(final_sample['user_id'][60])

In [32]:
for user_item in user_items:
    print(user_item)

Kryptonite
Un Simple Histoire
Love Letters
Passacaglia
Pastorale
Until The Morning
The Richest Man In Babylon
In The Waiting Line
Illumination
Big Yellow Taxi


In [33]:
recommendation.get_similar_items(['Oliver James - Fleet Foxes', 'The End - Pearl Jam'])

no. of unique songs in the training set: 4078
Non zero values in cooccurence_matrix :0


,user_id,song,score,rank
0,,Diary,0.0,1
1,,(When You Wake) You're Still In A Dream,0.0,2
2,,Bows & Arrows,0.0,3
3,,Mykonos,0.0,4
4,,Portlandtown,0.0,5
5,,Suesfine,0.0,6
6,,Look!,0.0,7
7,,Several Girls Galore,0.0,8
8,,All Together Now,0.0,9
9,,Groovin' In The Midnight,0.0,10


In [34]:
final_data['release'][6]

'Lift Me Up'

In [35]:
item_users = recommendation.get_item_users(final_sample['release'][1])

In [36]:
count=0
for user in item_users:
    count+=1

print('No of users: ',count)
for user in item_users:
  print(user)

No of users:  11
5495dccfae0b78a23c60c95db39e1e1d6e99ba6d
8061f61372876878c2d67bc49b3ddbd2c83d69e2
4fbde29be137e7179046c148f9e15db17c3278be
ab3883c6182f050d6bbca6f2cd8983eb81084417
a344d1f94e0f6d5783860f62d8bc8ba2fec3d530
248378ac27e1745d6a9d59392b7dc5b02a6186a6
93d158458cbf375218ae7de7fd428bf94cc1491e
ff8891b901eed1a3abc9bd4c53a17d2bcdc0ead7
fd50c4007b68a3737fe052d5a4f78ce8aa117f3d
d47c829c4ff6270f9ab82ca36b2805951b62a208
7c49f3e11a6fce79ae60b44df2a3edcf6eb2e163


In [37]:
# give song recommendation for that user
recommendation.recommend(final_sample['user_id'][5])

No. of unique songs for the user: 6
Non zero values in cooccurence_matrix :788


,user_id,song,score,rank
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,Sayonara-Nostalgia,0.099789,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,Revelry,0.074193,2
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,Représente,0.071636,3
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,Undo,0.060489,4
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,Rianna,0.059678,5
5,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,Secrets,0.057037,6
6,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,Die Kunst der Fuge_ BWV 1080 (2007 Digital Rem...,0.054335,7
7,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,Robot Soul (Radio Edit),0.053788,8
8,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,A Beggar On A Beach Of Gold,0.052697,9
9,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,I.D.G.A.F,0.048485,10
